In [32]:
from dotenv import load_dotenv
load_dotenv()

# Use the variable with:
import os
API_KEY = os.getenv("API_KEY")
STEAM_ID = os.getenv("STEAM_ID")

import requests
import pandas as pd
from tqdm.auto import tqdm

data_folder = "data"

General flow:
- Start with my Steam ID.
- Get a list of my games and save it in a dataframe.
- Everytime the dataframe is updated, save it in a csv file.
- Append the SteamID to the list of already visited SteamIDs.
- For each SteamID, get a list of their friends, check if they are already in the list of visited SteamIDs, and if not, add them to the list of SteamIDs to visit.
- Save the list of SteamIDs to visit in a csv file.
- Repeat the process.

In [33]:
# Takes a Steam ID and returns a DataFrame row with all of the user's games and playtime
def get_games(steam_id):
    url = f'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={API_KEY}&steamid={steam_id}&format=json&include_played_free_games=1'
    response = requests.get(url)
    data = response.json()
    data = data['response']
    # only extract app_id and playtime_forever from data['games']
    clean_data = []
    if 'games' not in data:
        return None, None
    for game in data['games']:
        if game['playtime_forever'] == 0:
            continue
        clean_data.append({'id': game['appid'], 'time': game['playtime_forever']})
    games = pd.DataFrame({'Total Games': data['game_count'], 'Played Games': len(clean_data), 'Games': [clean_data]}, index=[steam_id])
    return games

def get_games_2(steam_id):
    url = f'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={API_KEY}&steamid={steam_id}&format=json&include_played_free_games=1'
    response = requests.get(url)
    data = response.json()
    data = data['response']
    # only extract app_id and playtime_forever from data['games']
    # create empty games_data dataframe
    games_data = pd.DataFrame(columns=['steam_id', 'id', 'time'])

    #TODO: turn this into a list of dicts and simply use pd.DataFrame(list_of_dicts) instead of doing whatever the fuck I am doing currently

    if 'games' not in data:
        return None, None
    for game in data['games']:
        if game['playtime_forever'] == 0:
            continue
        new_row = pd.DataFrame({'steam_id': steam_id, 'id': game['appid'], 'time': game['playtime_forever']}, index=[steam_id])
        games_data = pd.concat([games_data, new_row], ignore_index=True)
    games = pd.DataFrame({'total_games': data['game_count'], 'played_games': len(games_data)}, index=[steam_id])
    
    return games, games_data


In [34]:
# Takes a Steam ID and returns a list of the user's friends
def get_friends(steam_id, visited_friends):
    url = f'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={API_KEY}&steamid={steam_id}&relationship=friend'
    response = requests.get(url)
    data = response.json()
    if 'friendslist' not in data:
        return None
    friends_to_visit = []
    for friend in data['friendslist']['friends']:
        if friend['steamid'] not in visited_friends:
            friends_to_visit.append(friend['steamid'])
    return friends_to_visit

In [35]:
# Saves all already visited friend IDs and saves it to a file
def save_visited_friends(visited_friends):
    visited_friends_file = os.path.join(data_folder, 'visited_friends.txt')
    with open(visited_friends_file, 'w') as f:
        for friend in visited_friends:
            f.write(f'{friend}\n')

# Saves the queue of friends to visit to a file
def save_friends_queue(friends_queue):
    friends_queue_file = os.path.join(data_folder, 'friends_queue.txt')
    with open(friends_queue_file, 'w') as f:
        for friend in friends_queue:
            f.write(f'{friend}\n')

# Appends the games dataframe to a csv file (or creates a new one if it doesn't exist)
def save_games(games, filename, overwrite=False):
    if not os.path.isfile(filename) or overwrite:
        games.to_csv(filename, header=['total_games', 'played_games'], index_label='steam_id')
    else:
        games.to_csv(filename, mode='a', header=False)
    
def save_game_time(games_data, filename, overwrite=False):
    if not os.path.isfile(filename) or overwrite:
        games_data.to_csv(filename, header=['steam_id', 'id', 'time'], index=False)
    else:
        games_data.to_csv(filename, mode='a', header=False, index=False)

In [40]:
RESTART = True # Set to True to start from scratch

# Read queue.txt if exists, otherwise start with STEAM_ID
queue_file = os.path.join(data_folder, 'friends_queue.txt')
if os.path.exists(queue_file) and not RESTART:
    with open(queue_file, 'r') as f:
        friends_queue = f.read().splitlines()
else:
    friends_queue = [STEAM_ID]

# Read visited_friends.txt if exists
visited_friends_file = os.path.join(data_folder, 'visited_friends.txt')
if os.path.exists(visited_friends_file) and not RESTART:
    with open(visited_friends_file, 'r') as f:
        visited_friends = f.read().splitlines()
else:
    visited_friends = []

user_games_data_file = os.path.join(data_folder, 'user_games_data.csv')
games_time_file = os.path.join(data_folder, 'games_time.csv')
if RESTART:
    if os.path.exists(user_games_data_file):
        os.remove(user_games_data_file)
    if os.path.exists(games_time_file):
        os.remove(games_time_file)

iterations = 10000

for i in tqdm(range(iterations)):
    user = friends_queue.pop(0)
    if user in visited_friends:
        continue
    visited_friends.append(user)

    if (i+1)%100 == 0:
        print(f"Queue: {len(friends_queue)} | Visited total: {len(visited_friends)}")

    # Get games and append them to the csv
    
    games, games_data = get_games_2(user)
    
    
    # Get (not yet visited) friends of the user and append them to the queue
    friends = get_friends(user, visited_friends)
    if friends is not None and len(friends) + len(friends_queue) < 3000:
        friends_queue.extend(friends)

    if games is not None and games['played_games'][0] > 0:
        save_games(games, user_games_data_file)
        save_game_time(games_data, games_time_file)

    # Save the visited friends and the friends queue
    save_visited_friends(visited_friends)
    save_friends_queue(friends_queue)   
    

  0%|          | 0/10000 [00:00<?, ?it/s]

Queue: 2982 | Visited total: 100
Queue: 2993 | Visited total: 200
Queue: 2985 | Visited total: 299


KeyboardInterrupt: 

In [19]:
def get_full_games_list():
    url = "http://api.steampowered.com/ISteamApps/GetAppList/v0002/"
    response = requests.get(url)
    data = response.json()
    games_list = pd.DataFrame(data['applist']['apps'])
    games_list.to_csv(os.path.join(data_folder, 'games_list.csv'), index=False)

# get_full_games_list()

In [38]:
def prepare_games_database(target):
    games_time_file = os.path.join(data_folder, 'games_time.csv')
    games_list = os.path.join(data_folder, 'games_list.csv')
    games_data = pd.read_csv(games_time_file)
    games_data = list(set(games_data['id']))
    games_list = pd.read_csv(games_list)
    games_name = []
    for game in games_data:
        game_name = games_list[games_list['appid'] == game]['name'].values
        if len(game_name) == 0:
            game_name = ['Unknown']
        games_name.append(game_name[0])
    games_data = pd.DataFrame({'id': games_data, 'name': games_name})
    games_data.to_csv(target, index=False)
    games_data.set_index('id', inplace=True)
    return games_data

game_names_file = os.path.join(data_folder, 'game_names.csv')
game_names = prepare_games_database(game_names_file)

In [39]:
game_names

,name
id,
360450,Kiai Resonance
688130,Pogostuck: Rage With Your Friends
65540,Unknown
262150,Vanguard Princess
10,Counter-Strike
...,...
2228210,Project Astra Dominium
327670,Ironcast
491510,Vanilla Bagel: The Roguelike


In [122]:
games_data

,steam_id,id,time
0,76561197962287503,10,3
1,76561197962287503,70,77
2,76561197962287503,220,125
3,76561197962287503,240,472
4,76561197962287503,80,13
...,...,...,...
3809,76561197962287503,2562730,8985
3810,76561197962287503,1485590,798
3811,76561197962287503,2017080,109
3812,76561197962287503,440,539


In [123]:
games = pd.read_csv('user_games_data.csv', index_col=0)

In [127]:
games.loc[games.total_games > 50]

,total_games,played_games
steam_id,,
76561198280830500,1156,887
76561197962287503,5624,3814


In [128]:
games.head()

,total_games,played_games
steam_id,,
76561198280830500,1156,887
76561197962287503,5624,3814


In [30]:
games_time = pd.read_csv(os.path.join(data_folder, 'games_time.csv'))

In [31]:
games_time.loc[games_time.steam_id == int(STEAM_ID)]

,steam_id,id,time
0,76561198280830500,4000,80782
1,76561198280830500,400,637
2,76561198280830500,20900,498
3,76561198280830500,22000,632
4,76561198280830500,17410,396
...,...,...,...
940,76561198280830500,2881650,99
941,76561198280830500,2923300,8
942,76561198280830500,2977660,1
943,76561198280830500,1422450,34


(numpy.int64, str)

Ideas:
- get recently played games, another recommendation based on that
- recommend the most popular games for user with no games
- recommend games with similar tag for user with barely any games